In [20]:
from dotenv import load_dotenv
from typing import Annotated

load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START,END
from langgraph.graph.message import add_messages

In [21]:
from langchain.chat_models import init_chat_model

In [22]:
llm = init_chat_model('google_genai:gemini-2.0-flash')


class State(TypedDict):
    messages: Annotated[list,add_messages]

def chatbot(state: State) -> State:
    return {"messages":[llm.invoke(state['messages'])]}

builder = StateGraph(State)
builder.add_node('chatbot_node',chatbot)

builder.add_edge(START,'chatbot_node')
builder.add_edge('chatbot_node',END)

graph = builder.compile()

In [23]:
message = {'role':'user','content':'Who walked on the moon for the first time ? Print only the name'}
response = graph.invoke({'messages':[message]})



response['messages']

[HumanMessage(content='Who walked on the moon for the first time ? Print only the name', additional_kwargs={}, response_metadata={}, id='505d94e1-be01-41d3-a82b-dce778df8503'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--93578d1d-a6e3-4522-83ae-9fbe124dd8e2-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [24]:
state = None
while True:
    in_message = input('You: ')
    if in_message.lower() in {'quit','exit'}:
        break
    if state is None:
        state: State ={
            'messages':[{'role':'user','content':in_message}]
            
        }
    else:
        state['messages'].append({'role':'user','content':in_message})

    state = graph.invoke(state)
    print('Bot:',state['messages'][-1].content)

You:  Who was the first person to walk on the moon?


Bot: Neil Armstrong was the first person to walk on the moon.


You:  which year?


Bot: Neil Armstrong first walked on the moon in **1969**.


You:  exit
